In [1]:
import pandas as pd
from utils import dataset_map

def calculate_acc(labels, pred_labels):
    acc = 0
    count = 0
    for (label, pred_label) in zip(labels, pred_labels):
        try: 
            if label == pred_label.lstrip().lower():
                acc += 1
        except:
            print(pred_label)
        count += 1
    return acc / count

def save_to_markdown(output_paths, md_output_path, output_names=None, true_col='label', skip_cols=['sentence']):
    print()
    if output_names is not None and len(output_names) != len(output_paths):
        raise ValueError("The length of output_names must match the length of output_paths.")
    
    results_all = {}
    
    for i, output_path in enumerate(output_paths):
        try:
            df = pd.read_csv(output_path)
            model_names = [name for name in df.columns if name not in skip_cols + [true_col]]
            results = {}
            for model in model_names:
                results[model] = calculate_acc(df[true_col], df[model])
            results_all[output_path] = results
        except Exception as e:
            print(f"Error processing {output_path}: {e}")
            continue

    # Generate Markdown table
    if output_names is not None:
        headers = output_names
    else:
        headers = results_all.keys()
    
    md_table = "| Model | " + " | ".join(headers) + " |\n"
    md_table += "|-------|" + "|".join(["-----" for _ in headers]) + "|\n"
    
    # Get unique model names across all files
    all_models = set()
    for results in results_all.values():
        all_models.update(results.keys())
    
    for model in all_models:
        row = f"| {model} | "
        for output_path in results_all.keys():
            accuracy = results_all[output_path].get(model, "N/A")
            if accuracy != "N/A":
                row += f"{accuracy:.4f} | "
            else:
                row += f"{accuracy} | "
        md_table += row + "\n"
    
    # Save to Markdown file
    try:
        with open(md_output_path, 'w') as f:
            f.write(md_table)
        print(f"Markdown table saved to {md_output_path}")
    except Exception as e:
        print(f"Error saving Markdown file: {e}")

In [2]:
output_paths = [dataset_map[key]['path'] for key in dataset_map]
output_names = [key for key in dataset_map]
md_output_path = 'results.md'
save_to_markdown(output_paths, md_output_path, output_names)


nan
nan
Markdown table saved to results.md
